# Answering Business Questions using SQL



We will use chinook database:

    Chinook Database
Chinook is a sample database available for SQL Server, Oracle, MySQL, etc. It can be created by running a single SQL script. Chinook database is an alternative to the Northwind database, being ideal for demos and testing ORM tools targeting single and multiple database servers.

    Sample Data
Media related data was created using real data from an iTunes Library. It is possible for you to use your own iTunes Library to generate the SQL scripts, see instructions below. Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.). Sales information is auto generated using random data for a four year period.

# Introduction and Schema Diagram

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db  #connect jupyter to DB

# Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
    FROM customer
    LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


# Selecting New Albums to Purchase

finding out which genres sell the most tracks in the USA and choose three most sold

In [4]:
%%sql
WITH customers_usa AS 
        (
            SELECT * FROM invoice
            WHERE billing_country = 'USA' 
        ),
    genres_count AS
        (
            SELECT g.name name, COUNT(*) total FROM customers_usa cu
            INNER JOIN invoice_line il ON il.invoice_id = cu.invoice_id
            INNER JOIN track t ON t.track_id = il.track_id
            INNER JOIN genre g ON g.genre_id = t.genre_id
            GROUP BY g.genre_id
            ORDER BY 2 DESC
        )
SELECT gc.name genre, gc.total number_of_tracks, ROUND((CAST(gc.total AS FLOAT)/(            
                                                        SELECT COUNT(*) FROM customers_usa cu
                                                        INNER JOIN invoice_line il ON il.invoice_id = cu.invoice_id
                                                        INNER JOIN track t ON t.track_id = il.track_id
                                                            )*100),2) percentage
        FROM genres_count gc;

 * sqlite:///chinook.db
Done.


genre,number_of_tracks,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following genres:

    Rock
    Alternative & Punk
    Metal
they account to 77% of sales


# Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We will analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [5]:
%%sql
SELECT e.first_name || " " || e.last_name employee_name,
        e.hire_date hire_date, e.birthdate birth_date,
        COUNT(c.customer_id) customers, ROUND(SUM(i.total),2) total_sales,
        ROUND(SUM(i.total)/COUNT(c.customer_id),2) per_customer
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1;

 * sqlite:///chinook.db
Done.


employee_name,hire_date,birth_date,customers,total_sales,per_customer
Jane Peacock,2017-04-01 00:00:00,1973-08-29 00:00:00,212,1731.51,8.17
Margaret Park,2017-05-03 00:00:00,1947-09-19 00:00:00,214,1584.0,7.4
Steve Johnson,2017-10-17 00:00:00,1965-03-03 00:00:00,188,1393.92,7.41


We can see Jane is better than others while she makes 8.17 per customer. While Margaret has same number of customers but only makes 7.4. Steve has lower number of customers which can be explained because he was hired last.

# Analyzing Sales by Country

for each country:

    total number of customers
    total value of sales
    average value of sales per customer
    average order value

In [6]:
%%sql
WITH country_sale AS 
     (
         SELECT
                CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"  
           ELSE c.country
       END AS country,
         count(DISTINCT c.customer_id) count,
         ROUND(SUM(i.total),2) total_sales,
         ROUND(SUM(i.total)/count(DISTINCT c.customer_id),2) average_sales,
         ROUND(SUM(i.total)/count(distinct c.customer_id) ,2) customer_lifetime_value
    FROM customer c
         LEFT JOIN invoice i ON c.customer_id = i.customer_id
     GROUP by 1
     )
SELECT
    country, count, total_sales, average_sales, customer_lifetime_value
FROM
    (
    SELECT
        country_sale.*,
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS sort
    FROM country_sale
    ORDER BY sort ASC, total_sales DESC
   );

 * sqlite:///chinook.db
Done.


country,count,total_sales,average_sales,customer_lifetime_value
USA,13,1040.49,80.04,80.04
Canada,8,535.59,66.95,66.95
Brazil,5,427.68,85.54,85.54
France,5,389.07,77.81,77.81
Germany,4,334.62,83.66,83.66
Czech Republic,2,273.24,136.62,136.62
United Kingdom,3,245.52,81.84,81.84
Portugal,2,185.13,92.57,92.57
India,2,183.15,91.57,91.57
Other,15,1094.94,73.0,73.0


# Albums vs Individual Tracks

In [7]:
%%sql
WITH invoice_tracks AS
    (
    SELECT invoice_id, track_id
        FROM invoice_line
    ),
  album_tracks AS
    (
    SELECT album_id, track_id
        FROM track t
        ORDER BY 1
    )
SELECT * FROM invoice_tracks it
    LEFT JOIN album_tracks at ON at.track_id= it.track_id;

 * sqlite:///chinook.db
Done.


invoice_id,track_id,album_id,track_id_1
21,13,1,13
23,1,1,1
23,6,1,6
23,7,1,7
23,8,1,8
23,9,1,9
23,10,1,10
23,11,1,11
23,12,1,12
23,13,1,13


In [8]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674



Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue

## Future Questions that can be answered:

    Which artist is used in the most playlists?
    How many tracks have been purchased vs not purchased?
    Is the range of tracks in the store reflective of their sales popularity?
    Do protected vs non-protected media types have an effect on popularity?